This notebook has the following components
1. It creates a YML file for each of the files in the macros directory you specify. Simplifying the process of copying over your macros to the docs site.
2. It takes the codegen output that you copied into a schema.yml file in your client project and populates the YML files with their corresponding codegen output. 

In [2]:
import os
import shutil
import ruamel.yaml

In [ ]:
macros_dir = '/path/to/sql/files' #this is the folder where your new macros models are
docs_dir = '/path/to/yml/files' #this is the folder where your new yml files will be created

In [ ]:
# Iterate over all files in the source directory
for filename in os.listdir(macros_dir):
    if filename.endswith('.sql'):
        # Create a new filename with a .yml extension
        new_filename = f'{os.path.splitext(filename)[0]}.yml'

        # Create a new empty file with the new filename in the destination directory
        with open(os.path.join(docs_dir, new_filename), 'w'):
            pass

        # Copy the .sql file to the destination directory with the new filename and extension
        shutil.copy2(os.path.join(macros_dir, filename),
                     os.path.join(macros_dir, new_filename))

PAUSE! Before you run the following cells, you need to do the following:
1. Go to your client project and run `dbt run-operation generate_model_yml --args '{"model_name": "model_name"}'` for all of your models. 
2. Copy the output to a schema.yml file in your client project.

In [1]:
#enter filepath for codegen output
codegen_output = "/Users/frydaguedes/Projects/bsd-nyccfb/documentation/schema.yml"

In [3]:
yaml = ruamel.yaml.YAML()
yaml.indent(mapping=4, sequence=4, offset=2)
yaml.preserve_quotes = True
with open(codegen_output, 'r') as f:
    content = f.read()
    codegen_yml = yaml.load(content)

In [4]:
dict_of_models = {model['name']: model for model in codegen_yml['models']}

In [5]:
for root, dirs, files in os.walk('../documentation/'):
    for file in files:
        name, extension = os.path.splitext(file)
        if extension == '.yml' and name in dict_of_models:
            filepath = os.path.join(root, file)
            try:
                file_yml = ruamel.yaml.comments.CommentedMap()
                model = dict_of_models[name]
                file_yml['models'] = [model]
                with open(filepath, 'w') as f:
                    yaml.dump(file_yml, f)
            except Exception as e:
                print(f"Error processing file {filepath}: {e}")